In [ ]:
## VIZU OPTIMISATION RESULTS ##
import glob 
import numpy as np
import sys
import os
import plotly.graph_objects as go
from skopt import load
path = os.path.abspath('..')
sys.path.append(path+"/pkg/")

import save_utils

In [ ]:

x_temp ='Size {0} <br> K {1}'
models = ['SNN','SNNrand','SNNsm','SNNnm','VarSmile','BOCPA']

colors = [(255,0,0),(32,12,128),(0,167,159),(0,116,128),(65,61,68),(202,199,199)]
colors += [(255,0,0),(0,116,128),(65,61,68),(202,199,199),(163,133,199)]
paths = [os.path.abspath('..')]
fig = go.Figure()
m = 0
np.random.seed()
for path in paths:
    for model in models:
        x = []
        y_std = []
        y = []
        for filename in sorted(glob.glob(path+'/results/optimisation/results_optimization_{0}_'.format(model)+'*.pkl')):
            print(filename)
            params = save_utils.find_parameters(filename.split('/')[-1]) # moves,rooms

            try:
                with open(filename.replace('.npy','.pkl'),'rb') as file:
                    results = load(file)
                    y += [int(2 * params[0]) * params[1] * results.fun]
                    print(model,*params, results.x,results.fun)
            except:
                results = np.load(filename,allow_pickle = True).item()
                y += [int(2 * params[0])*params[1] * np.mean(results['error'])]
            
            x += [x_temp.format(int(params[1]),int(2 * params[0]))]
            
                    
                
        idx = sorted(range(len(x)), key=lambda k: x[k])
        fig.add_trace(go.Histogram(histfunc="sum", y=np.array(y)[idx], x=np.array(x)[idx], name="{0}".format(model), marker_color='rgb{0}'.format(colors[models.index(model)])))
fig.update_layout(
#     width=400, height=600,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=.8, y=1.),
    showlegend= True,
    font=dict(
            family="sans-serif",
            size=20,
            color="black"
    )
)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
# fig.write_image("error_comparison_varSMILE.pdf") uncomment if you have psutils
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
colors = [(255,0,0),(32,12,128),(0,167,159),(0,116,128),(65,61,68),(202,199,199)]
m = 0
x_temp ='Size {0} <br> K {1}'
df = pd.read_csv (path+'/results/optimisation/full_results.csv')
fig = go.Figure()
factor = np.array([2 * 16,4* 16,8* 16,2* 32,4* 32,8* 32])
for model in df.columns[~ df.columns.str.contains('^Unnamed')].values:
    print(model)
    
    errors = np.array(df[model][df[df[model]=='mean'].index.values+1].values) #* factor
    errors = np.array([float(error) for error in errors ]) * factor
    stds = df.iloc[:,df.columns.get_loc(model)+1][df[df[model]=='mean'].index.values+1].values
    stds = np.array([float(std) for std in stds ]) * (factor)
    x = [x_temp.format(int(value.split('_')[0]),2 * int(value.split('_')[1])) for value in df[model].values[:-3:5]]                          
    idx = sorted(range(len(x)), key=lambda k: x[k])
    fig.add_trace(go.Histogram(histfunc="sum", y=errors[idx], x=np.array(x)[idx],error_y = dict(type='data', array=stds), name="{0}".format(model), marker_color='rgb{0}'.format(colors[m])))
    m += 1
fig.update_layout(
#     width=400, height=600,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=.0, y=1.),
    showlegend= False,
    font=dict(
            family="sans-serif",
            size=20,
            color="black"
    )
)
fig.write_image("error_comparison.pdf") #uncomment if you have psutils

fig.show()